# Browser observability 및 세션 재생 

## 개요

이 튜토리얼에서는 Agentcore Browser 세션에 observability를 추가하고 브라우저 콘솔 로그, 네트워크 로그, 전송된 CDP 명령 및 브라우저에서 수행된 agent 작업을 확인하는 방법을 배웁니다. 


### 튜토리얼 세부 정보


| 정보                | 세부 사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형        | 대화형                                                                            |
| Agent 유형          | 단일                                                                              |
| Agentic Framework   | Nova Act                                                                         |
| LLM model           | Amazon Nova Act model                                                            |
| 튜토리얼 구성 요소   | Agentocre Browser 콘솔에서 브라우저 로그 관찰                                      |
| 튜토리얼 분야        | vertical                                                                         |
| 예제 복잡도          | 쉬움                                                                              |
| 사용된 SDK          | Amazon Bedrock AgentCore Python SDK, boto3 SDK, Nova Act                          |

### 튜토리얼 아키텍처

이 튜토리얼에서는 브라우저 콘솔 로그, 네트워크 로그, CDP 명령 및 브라우저에서 수행된 agent 작업을 관찰하는 방법을 살펴봅니다.   


### 튜토리얼 주요 기능

* browser tool에 대한 세션 녹화 활성화 
* browser tool과 함께 Nova Act 사용
* 로그 및 세션 재생 관찰

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Amazon Boto3 SDK
* Nova Act SDK 및 API 키 - https://nova.amazon.com/act로 이동하여 API 키를 생성하세요

In [ ]:
!pip install  -r requirements.txt --quiet

## 녹화가 활성화된 커스텀 AgentCore Browser 리소스 생성
기본 browser tool에는 녹화 기능이 켜져 있지 않으므로 먼저 녹화가 활성화된 browser tool 리소스를 생성해야 합니다. 그런 다음 이 browser 리소스를 사용하여 브라우저 세션을 시작합니다. 

In [ ]:
## Create an S3 bucket to store browser recordings
## If you want to use an existing bucket, skip this step and update the bucket name in the next step.
import boto3
import uuid
from boto3.session import Session

boto_session = Session()

region = boto_session.region_name
s3_client = boto3.client('s3', region_name=region)

# 고유한 버킷 이름 생성 (UUID 앞 8자리 사용)
bucket_name = f"agentcore-browser-recordings-{str(uuid.uuid4())[:8]}"
# 리전별 LocationConstraint 설정 필요
s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
print(f"Created S3 bucket: {bucket_name}")

### browser tool 리소스 생성에 필요한 권한을 가진 실행 역할 생성

적절한 권한을 가진 실행 역할을 생성하기 위한 헬퍼 유틸리티를 만들어 보겠습니다. 

In [ ]:
## Create execution role with permisions to create browser 
def create_agentcore_role(agent_name):
    iam_client = boto3.client('iam')
    agentcore_role_name = f'agentcore-{agent_name}-role'
    boto_session = Session()
    region = boto_session.region_name
    # STS를 통해 현재 AWS 계정 ID 조회
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    
    # Browser tool 사용에 필요한 권한 정책 정의
    role_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "BrowserPermissions",
                "Effect": "Allow",
                "Action": [
                    "bedrock-agentcore:ConnectBrowserAutomationStream",
                    "bedrock-agentcore:ListBrowsers",
                    "bedrock-agentcore:GetBrowserSession",
                    "bedrock-agentcore:ListBrowserSessions",
                    "bedrock-agentcore:CreateBrowser",
                    "bedrock-agentcore:StartBrowserSession",
                    "bedrock-agentcore:StopBrowserSession",
                    "bedrock-agentcore:ConnectBrowserLiveViewStream",
                    "bedrock-agentcore:UpdateBrowserStream",
                    "bedrock-agentcore:DeleteBrowser",
                    "bedrock-agentcore:GetBrowser"
                ],
                "Resource": "*"
            },
            {
                "Sid": "S3Permissions",
                "Effect": "Allow",
                "Action": [
                    "s3:PutObject",
                    "s3:GetObject",
                    "s3:ListBucket"
                ],
                # 녹화 데이터 저장을 위한 S3 버킷 접근 권한
                "Resource": [
                    f"arn:aws:s3:::{bucket_name}",
                    f"arn:aws:s3:::{bucket_name}/*"
                ]
            },
            {
                "Sid": "CloudWatchLogsPermissions",
                "Effect": "Allow",
                "Action": [
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents",
                    "logs:DescribeLogStreams"
                ],
                "Resource": "*"
            }
        ]
    }
    # bedrock-agentcore 서비스가 이 역할을 assume할 수 있도록 신뢰 정책 설정
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "AssumeRolePolicy",
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock-agentcore.amazonaws.com"
                },
                "Action": "sts:AssumeRole",
                # 보안을 위해 동일 계정 및 리전의 agentcore 리소스만 허용
                "Condition": {
                    "StringEquals": {
                        "aws:SourceAccount": f"{account_id}"
                    },
                    "ArnLike": {
                        "aws:SourceArn": f"arn:aws:bedrock-agentcore:{region}:{account_id}:*"
                    }
                }
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(
        assume_role_policy_document
    )
    role_policy_document = json.dumps(role_policy)

    try:
        # Create IAM Role with the trust policy
        agentcore_iam_role = iam_client.create_role(
            RoleName=agentcore_role_name,
            AssumeRolePolicyDocument=assume_role_policy_document_json,
        )
        print(f"Role {agentcore_role_name} created successfully.")

        # Attach the inline permissions policy to the role
        iam_client.put_role_policy(
            RoleName=agentcore_role_name,
            PolicyName=f'{agentcore_role_name}-inline-policy',
            PolicyDocument=role_policy_document
        )
        print(f"Inline policy attached to role {agentcore_role_name}.")

    except iam_client.exceptions.EntityAlreadyExistsException:
        print("Role already exists -- deleting and creating it again")
        
        # 기존 inline policy 모두 삭제
        policies = iam_client.list_role_policies(RoleName=agentcore_role_name)
        for policy_name in policies['PolicyNames']:
            iam_client.delete_role_policy(
                RoleName=agentcore_role_name,
                PolicyName=policy_name
            )
        
        # Delete and re-create the role
        print(f"Deleting role {agentcore_role_name}...")
        iam_client.delete_role(RoleName=agentcore_role_name)
        print(f"Recreating role {agentcore_role_name}...")
        
        agentcore_iam_role = iam_client.create_role(
            RoleName=agentcore_role_name,
            AssumeRolePolicyDocument=assume_role_policy_document_json
        )
        print(f"Role {agentcore_role_name} recreated successfully.")

        # Re-attach the inline permissions policy to the re-created role
        iam_client.put_role_policy(
            RoleName=agentcore_role_name,
            PolicyName=f'{agentcore_role_name}-inline-policy',
            PolicyDocument=role_policy_document
        )
        print(f"Inline policy re-attached to role {agentcore_role_name}.")

    # IAM 변경사항이 전파될 때까지 대기
    time.sleep(10)
    
    return agentcore_iam_role

### 녹화가 활성화된 browser tool 리소스 생성

In [ ]:
## Use boto3 to create a custom browser tool resource with recording enabled
import boto3
import time
import json
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

# bedrock-agentcore-control 클라이언트 생성
cp_client = boto3.client('bedrock-agentcore-control', region_name=region)

# Define parameters for the Browser Tool
browser_name = "my_custom_browser"
browser_description = "Test browser for observability and session replay"
execution_role = create_agentcore_role(browser_name)
execution_role_arn = execution_role['Role']['Arn'] # Todo: Replace with your IAM role ARN
s3_bucket_name = bucket_name # Replace with your S3 bucket name if you have an existing bucket
s3_prefix = "replay-data"

try:
    response = cp_client.create_browser(
        name=browser_name,
        description=browser_description,
        networkConfiguration={
            "networkMode": "PUBLIC" # Or "VPC" if you need VPC integration
        },
        executionRoleArn=execution_role_arn,
        clientToken=str(uuid.uuid4()), # Unique token for idempotency
        # 세션 녹화 활성화 및 S3 저장 위치 지정
        recording={
            "enabled": True,
            "s3Location": {
                "bucket": s3_bucket_name,
                "prefix": s3_prefix
            }
        }
    )
    print(response)
    print(f"Successfully created Browser Tool: {response['browserId']}")
    browserId = response['browserId']
except cp_client.exceptions.ConflictException as e:
    print("Browser Tool with this name already exists. Please choose a different name.")


## Nova Act 스크립트 생성
Nova Act는 이전 단계에서 생성한 browser tool 리소스를 사용하여 브라우저 세션을 시작하고 그 위에서 브라우저 작업을 실행합니다.

In [ ]:
%%writefile basic_browser_with_nova_act.py
"""Browser automation script using Amazon Bedrock AgentCore and Nova Act.

This script demonstrates AI-powered web automation by:
- Initializing a browser session through Amazon Bedrock AgentCore
- Connecting to Nova Act for natural language web interactions
- Performing automated searches and data extraction using browser
"""

from bedrock_agentcore.tools.browser_client import browser_session , BrowserClient
from nova_act import NovaAct
from rich.console import Console
import argparse
import json

console = Console()

from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print("using region", region)

def browser_with_nova_act(prompt, starting_page, nova_act_key,  browserId, region="us-west-2"):
    result = None
    
    # BrowserClient 생성 및 세션 시작
    browser_client = BrowserClient(region)
    browser_client.start(identifier=browserId) # Use the created browser tool ID here
    
    # WebSocket URL과 헤더 생성 (CDP 연결용)
    ws_url, headers = browser_client.generate_ws_headers()
    try:
        # Nova Act를 사용하여 브라우저 자동화 실행
        with NovaAct(
            cdp_endpoint_url=ws_url,
            cdp_headers=headers,
            nova_act_api_key=nova_act_key,
            starting_page=starting_page,
        ) as nova_act:
            result = nova_act.act(prompt)
    except Exception as e:
        console.print(f"NovaAct error: {e}")

    finally:
        # 브라우저 세션 종료 (녹화 저장)
        browser_client.stop()
        return result


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--prompt", required=True, help="Browser Search instruction")
    parser.add_argument("--starting-page", required=True, help="Starting URL")
    parser.add_argument("--nova-act-key", required=True, help="Nova Act API key")
    parser.add_argument("--region", default="us-west-2", help="AWS region")
    parser.add_argument("--browserID", required=True, help="Browser Tool ID to use")
    args = parser.parse_args()

    result = browser_with_nova_act(
        args.prompt, args.starting_page, args.nova_act_key, args.browserID, args.region
    )
    console.print(f"\n[cyan] Response[/cyan] {result.response}")
    console.print(f"\n[bold green]Nova Act Result:[/bold green] {result}")

#### 스크립트 실행
스크립트를 실행하기 전에 아래에 Nova Act API 키를 붙여넣으세요. 

In [ ]:
NOVA_ACT_KEY= '' ### Paste your Nova Act Key here

In [ ]:
!python basic_browser_with_nova_act.py --prompt "Search for macbooks and extract the details of the first one" --starting-page "https://www.amazon.com/" --browserID {browserId} --nova-act-key {NOVA_ACT_KEY}

## Agentcore Browser Console에서 Observability
* 스크립트가 실행되는 동안 AWS 콘솔로 이동할 수 있습니다: https://us-west-2.console.aws.amazon.com/bedrock-agentcore/builtInTools
그리고 "Browser use tools" 탭을 클릭하세요. 다른 리전에서 실행 중인 경우 이 URL의 리전을 변경하세요.
* "my-custom-browser"를 클릭하세요. 세션이 아직 실행 중이면 라이브 뷰를 볼 수 있는 링크가 표시되고, 그렇지 않으면 녹화를 볼 수 있는 링크가 표시됩니다. 
* 세션이 실행 중이면 종료될 때까지 기다린 다음 녹화 보기를 클릭하세요. 
다음과 유사한 페이지가 표시됩니다

![image](./images/browser_recording_1.png)

* #### 이제 녹화된 브라우저 세션을 재생할 수 있습니다 
* #### 세션 중에 방문한 각 페이지를 검사할 수 있습니다 
* #### Action 탭에서 agent가 수행한 작업을 확인할 수 있습니다 
* #### Page DOM 세부 정보, Console 로그, 브라우저로 전송된 CDP 명령 및 Network 로그를 확인할 수 있습니다 
* #### 추가 디버깅을 위해 각 로그를 다운로드할 수 있습니다
* #### Actions 탭에서 각 작업에 대해 "View"를 클릭하여 브라우저에서 수행된 정확한 작업을 확인할 수 있습니다 
* #### 예를 들어: "Click" 작업 유형 중 하나를 보고 브라우저의 빨간색 원을 확인하세요. 이것은 클릭 작업이 발생한 정확한 위치를 나타냅니다. 

# 축하합니다. 즐거운 탐험 되세요! 